In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import os as os
from PIL import Image
from scipy import misc
import matplotlib.pyplot as plt
import shutil
%matplotlib inline
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [2]:
imagenet_data = torchvision.datasets.ImageFolder(root='./train/',transform=transforms.Compose([transforms.ToTensor()]))

In [3]:
train_loader = torch.utils.data.DataLoader(dataset=imagenet_data, 
                                           batch_size=100, 
                                           shuffle=True)

In [4]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)
        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)
        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [5]:
input_dim = 1280*1918
hidden_dim = 1000
output_dim = 318

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [6]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [7]:
print(model.parameters())

print(len(list(model.parameters())))

# FC 1 Parameters 
print(list(model.parameters())[0].size())

# FC 1 Bias Parameters
print(list(model.parameters())[1].size())

# FC 2 Parameters
print(list(model.parameters())[2].size())

# FC 2 Bias Parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x000001D4358611A8>
4
torch.Size([1000, 2455040])
torch.Size([1000])
torch.Size([318, 1000])
torch.Size([318])


In [8]:
iter = 0
for epoch in range(20):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 2455040).requires_grad_()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        print(loss)
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1

ValueError: Expected input batch_size (300) to match target batch_size (100).